In [ ]:
import matplotlib.pyplot as plt
import DMK_go_coude as Fns
import numpy as np
import os, readcol
import scipy.optimize as optim
import scipy.interpolate as interp
import pickle

from astropy.io import fits 
from mpfit import mpfit
from scipy import signal

%matplotlib inline

In [ ]:
dir = os.getenv("HOME") + '/Research/YMG/coude_data/20140321/'
rdir = dir + 'reduction/'
codedir = os.getenv("HOME") + '/codes/coudereduction/'
#codedir = os.getenv("HOME") + '/Research/Codes/coudereduction/'

In [ ]:
os.chdir(dir)

In [ ]:
DarkCurVal = 0.0

InfoFile = 'headstrip.csv'
FileInfo = readcol.readcol( InfoFile, fsep = ',', asRecArray = True )
DarkCube = FileInfo.ExpTime * DarkCurVal

In [ ]:
BiasInds = np.where( FileInfo.Type == 'zero' )[0]
FlatInds = np.where( FileInfo.Type == 'flat' )[0]
ArcInds  = np.where( (FileInfo.Type == 'comp') & ( (FileInfo.Object == 'Thar') | (FileInfo.Object == 'THAR') | (FileInfo.Object == 'A') ) )[0]
ObjInds  = np.where( (FileInfo.Type == 'object') & (FileInfo.Object != 'SolPort') & (FileInfo.Object != 'solar port') & (FileInfo.Object != 'solar_ort') )[0]

In [ ]:
CalsDone = True
SuperBias, FlatField = Fns.Basic_Cals( FileInfo.File[BiasInds], FileInfo.File[FlatInds], CalsDone, rdir, plots = False )

In [ ]:
ShowBPM = False
BPM = Fns.Make_BPM( SuperBias, FlatField, 99.9, ShowBPM )

In [ ]:
RdNoise  = FileInfo.rdn[ArcInds] / FileInfo.gain[ArcInds]
DarkCur  = DarkCube[ArcInds] / FileInfo.gain[ArcInds]
ArcCube, ArcSNR = Fns.Make_Cube( FileInfo.File[ArcInds], RdNoise, DarkCur, Bias = SuperBias )

RdNoise  = FileInfo.rdn[ObjInds] / FileInfo.gain[ObjInds]
DarkCur  = DarkCube[ObjInds] / FileInfo.gain[ObjInds]
ObjCube, ObjSNR = Fns.Make_Cube( FileInfo.File[ObjInds], RdNoise, DarkCur, Bias = SuperBias, Flat = FlatField, BPM = BPM )

In [ ]:
OrderStart = -32
TraceDone = True
MedCut = 95.0
MedTrace, FitTrace = Fns.Get_Trace( FlatField, ObjCube, OrderStart, MedCut, rdir, TraceDone, plots = False )

In [ ]:
#fullspec, fullsig_spec = Fns.extractor( ObjCube, ObjSNR, FitTrace, quick = False, nosub = False, arc = False )

In [ ]:
spec     = pickle.load(open(rdir+'extracted_spec_oldway.pkl','rb'))
sig_spec = pickle.load(open(rdir+'extracted_sigspec_oldway.pkl','rb'))
wspec     = pickle.load(open(rdir+'extracted_wspec_oldway.pkl','rb'))
sig_wspec = pickle.load(open(rdir+'extracted_sigwspec_oldway.pkl','rb'))

In [ ]:
wspec      = wspec[:,::-1,:]
sig_wspec  = sig_wspec[:,::-1,:]
spec       = spec[:,::-1,:]
sig_spec   = sig_spec[:,::-1,:]

In [ ]:
def Gaussian( x, A, mean, sigma, const ):
    gauss = A * np.exp( - ( x - mean ) ** 2.0 / ( 2.0 * sigma ** 2 ) ) + const
    return gauss
    
def TwoGaussian( x, A1, mean1, sigma1, const, A2, mean2, sigma2 ):
    return Gaussian( x, A1, mean1, sigma1, const ) + Gaussian( x, A2, mean2, sigma2, const )

In [ ]:
THAR      = fits.open( codedir + 'thar_photron.fits' )[0]
THARhead  = THAR.header
THARspec  = THAR.data
THARwav   = np.arange( len( THARspec ) ) * THARhead['CDELT1'] + THARhead['CRVAL1']
THARlines = readcol.readcol( codedir + 'ThAr_list.txt', asRecArray = True )

logTHARspec = np.log10(THARspec)

#qwe            = np.where((THARwav >= np.min(roughwav-10)) & (THARwav <= np.max(roughwav+10)))[0] 
#ls             = THARspec[qwe] - np.min(THARspec[qwe])
#region         = np.where(ls < np.median(ls))
#ls[region]     = np.median(ls)

In [ ]:
roughdisp  = 7000.0 / 60000.0 / 2.0
wavestart  = 6957.0
startorder = wspec.shape[1] - 15 - 1
arcspec    = wspec[0,startorder,:]
roughwav   = wavestart + roughdisp * np.arange( len( arcspec ) )

arcspec           = arcspec - np.min( arcspec )
belowmed          = np.where( arcspec < np.median( arcspec ) )
arcspec[belowmed] = np.median( arcspec )
logarcspec        = np.log10( arcspec )
logarcspec        = logarcspec - np.min( logarcspec )

testspec    = wspec[0,startorder,:]
logtestspec = np.log10( testspec )

In [ ]:
peaks = signal.find_peaks_cwt( arcspec, np.arange( 1, 2 ), min_snr = 1000, noise_perc = 20 )
print peaks
print roughwav[1928]
plt.plot( logarcspec, 'k-' )
for peak in peaks:
    plt.axvline( x = peak, color = 'r', ls = ':' )
plt.xlim( 1900, 2050 )
plt.show()

In [ ]:
def Find_Peaks_Wavelet( wav, spec, peaksnr = 5, pwidth = 10, minsep = 1 ):
    
    # Find peaks using the cwt routine from scipy.signal
    peaks = signal.find_peaks_cwt( spec, np.arange( 1, 2 ), min_snr = peaksnr, noise_perc = 20 )
    
    # Offset from start/end of spectrum by some number of pixels
    peaks = peaks[ (peaks > pwidth) & (peaks < len(spec) - pwidth) ]
    
    pixcent = np.array([])
    wavcent = np.array([])
        
    for peak in peaks:
        
        xi   = wav[peak - pwidth:peak + pwidth]
        yi   = spec[peak - pwidth:peak + pwidth]
        inds = np.arange( len(xi), dtype = float )
        
        pguess   = [ yi[9], np.median( inds ), 0.9, np.median( spec ) ]
        lowerbds = [ 0.1*pguess[0], pguess[1] - 2.0, 0.3, 0.0  ]
        upperbds = [ np.inf, pguess[1] + 2.0, 1.5, np.inf ]
        
        try:
            params, pcov = optim.curve_fit( Gaussian, inds, yi, p0 = pguess, bounds = (lowerbds,upperbds) )
            
            pixval  = peak - pwidth + params[1]
            pixcent = np.append( pixcent, pixval )
            
            ceiling = np.ceil( pixval ).astype(int)
            floor   = np.floor( pixval ).astype(int)
            slope   = ( wav[ceiling] - wav[floor] ) / ( ceiling - floor )
            wavval  = wav[floor] + slope * ( pixval - floor )
            wavcent = np.append( wavcent, wavval )
            
        except RuntimeError:
            pixval  = 'nan'
            
    vals = spec[pixcent.astype(int)]
    oks  = np.ones( len(pixcent), int )
    
    for i in range( len(wavcent) ):
        dist  = np.absolute( wavcent - wavcent[i] )
        close = np.where( dist <= minsep )[0]
        small = np.where( vals[close] < np.max( vals[close] ) )[0]
        if len(small) != 0: oks[close[small]] = -1
    keep    = np.where( oks == 1 )
    pixcent = pixcent[keep]
    wavcent = wavcent[keep]
            
    return pixcent, wavcent

def Get_WavSol( wav, spec, snrarr, pwidth = 10, minsep = 1 ):
    
    wavsols    = np.zeros( ( len(snrarr) + 1, len(spec) ) )
    wavsols[0] = wav
    
    for i in range( len(snrarr) ):
        
        pixcent, wavcent = Find_Peaks_Wavelet( wavsols[i], spec, peaksnr = snrarr[i] )
        
        THARthisord = [ line for line in THARlines.wav if line > wavsols[i,0] and line < wavsols[i,-1] ]
        
        matchedpix   = np.array([])
        matchedwav   = np.array([])
        matchedlines = np.array([])
        
        for j in range( len(wavcent) ):
            dists    = np.absolute( THARthisord - wavcent[j] )
            mindist  = np.min( dists )
            mindisti = np.argmin( dists )
            
            if mindist <= 1.5:
                matchedpix   = np.append( matchedpix, pixcent[j] )
                matchedwav   = np.append( matchedwav, wavcent[j] )
                matchedlines = np.append( matchedlines, THARthisord[mindisti] )

        wavspl = interp.InterpolatedUnivariateSpline( matchedpix, matchedlines, k = 1 )
        wavsol = wavspl( np.arange( len(spec) ) )
        
        wavsols[i+1] = wavsol
        
    return wavsols, matchedpix

In [ ]:
wavsols, matchedpix = Get_WavSol( roughwav, arcspec, [5] )

plt.plot( wavsols[-1], logarcspec, 'k-' )
plt.plot( wavsolspl, logarcspec, 'm--' )
plt.plot( THARwav, logTHARspec, 'r-' )
for peak in matchedpix:
    plt.axvline( x = wavsols[-1][peak.round().astype(int)], color = 'b' )
plt.xlim( 7050, 7080 )
plt.show()

In [ ]:
thisord = np.where( ( THARwav >= roughwav[0] - 5 ) & ( THARwav <= roughwav[-1] + 5 ) )
thisTHARwav  = THARwav[thisord]
thisTHARspec = THARspec[thisord]

peaks = signal.find_peaks_cwt( thisTHARspec, np.arange( 1, 2 ), min_snr = 5, noise_perc = 20 )
print len(peaks)
plt.clf()
plt.plot( thisTHARwav, np.log10(thisTHARspec), 'k-' )
for peak in peaks:
    plt.axvline( x = thisTHARwav[peak], color = 'r', ls = ':' )
#plt.xlim( 6964, 6967 )
plt.show()